# Pre-class Work

Speaker identification
​

There are three people in a room. Each says about 10 phonemes, before being randomly interrupted by someone else. When they speak they all sound the same, however each person tends to use different phonemes in their speech. Specifically we can model the following transition probabilities that someone will interrupt the current speaker: P(speaker i at time t+1 | speaker j at time t). We can also model the probability over phonemes given a particular speaker: P(phoneme | speaker i). The phonemes are identical to the ones introduced in problem 1 (but the transition matrices are obviously different, since they take a different form altogether).

+ Write down the update equations that you will need to train a hidden Markov model. 
+ Using the information given above, write down a sensible initialization for the transition matrix.
+ Write your own python code to train a hidden Markov model on the data. You may look at code online, but will need to reference any code that helps you with your implementation.
+ From matplotlb use a stackplot (https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.stackplot.html) to show the probability of a particular person speaking.
+ Audio dataset: https://course-resources.minerva.kgi.edu/uploaded_files/mke/VW8Rjr/speaker.wav.zip
+ Symbol dataset: https://course-resources.minerva.kgi.edu/uploaded_files/mke/n705lY/speaker

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = open("speaker.txt").read()



['A', 'p', 'o', 'g', 'e', 'k', 't']

'eoggeggAeg'

In [101]:
#i wrote this function but the helper functions, i modified from code here: 
#https://sambaiga.github.io/2017/05/03/hmm-intro.html
def baum_welch(data, prior = 1/7):
    unique_characters = list(set(data))
    transition = np.random.rand(7,7)
    emission = np.random.rand(7,7)
    for i in range(len(data)):
        alpha = forward(transition, emission, data, prior)
        beta = backward(transition, emission, data)
        g = gamma(transition, emission, data, prior)
        e = epsilon(transition, emission, data, prior)
        g2 = gamma(transition, emission, data, prior)
        transition = tr(e, g)
        emission = em(g2, g)
    return transition, emission
    
    

In [78]:
dict([(i[1], i[0]) for i in list(enumerate(unique_characters))])

{'A': 0, 'p': 1, 'o': 2, 'g': 3, 'e': 4, 'k': 5, 't': 6}

In [134]:
#modified code from 

def forward(A, B, obs_seq, prior):
    unique_characters = list(set(obs_seq))
    dic = dict([(i[1], i[0]) for i in list(enumerate(unique_characters))])
    T = len(obs_seq)
    N = A.shape[0]
    alpha = np.zeros((T, N))
    alpha[0] = prior*B[:, dic[obs_seq[0]]]
    for t in range(1, T):
        alpha[t] = alpha[t-1].dot(A) * B[:, dic[obs_seq[t]]]
    return alpha
    
def backward(A, B, obs_seq):
    unique_characters = list(set(obs_seq))
    dic = dict([(i[1], i[0]) for i in list(enumerate(unique_characters))])
    N = A.shape[0]
    T = len(obs_seq)

    beta = np.zeros((N,T))
    beta[:,-1:] = 1

    for t in reversed(range(T-1)):
        for n in range(N):
            beta[n,t] = np.sum(beta[:,t+1] * A[n,:] * B[:, dic[obs_seq[t+1]]])
            

    return beta

def gamma(A, B, obs_seq, prior):
    alpha = forward(A, B, obs_seq, prior)
    beta  = backward(A, B, obs_seq)
    obs_prob = alpha[-1].sum()
    #print(alpha.shape)
    return (np.dot(alpha,beta) / obs_prob)


def epsilon(A, B, obs_seq, prior):
    alpha = forward(A, B, obs_seq, prior)
    beta  = backward(A, B, obs_seq)
    obs_prob = alpha[-1].sum()
    return (np.dot(np.dot(np.dot(alpha,A), B), beta)) / obs_prob

def tr(e, g):
    return e/g
    
def em(g1, g2):
    return g1/g2
    

In [135]:
# unique_characters = list(set(data))
# transition = np.random.rand(7,7)
# emission = np.random.rand(7,7)
# for i in range(len(data)):
#     alpha = forward(transition, emission, data, 1/7)
#     beta = backward(transition, emission, data)
#     g = gamma(transition, emission, data, 1/7)#alpha, beta, emission, transition)
# #     e = epsilon(alpha, beta, emission, transition)
# #     g2 = gamma(alpha, beta, emission, transition)
# #     transition = tr(e, g)
# #     emission = em(g2/g)
# # return transition, emission

In [141]:
def forward(params, observations):
    pi, A, O = params
    N = len(observations)
    S = pi.shape[0]
    
    alpha = np.zeros((N, S))
    
    # base case
    alpha[0, :] = pi * O[:,observations[0]]
    
    # recursive case
    for i in range(1, N):
        for s2 in range(S):
            for s1 in range(S):
                alpha[i, s2] += alpha[i-1, s1] * A[s1, s2] * O[s2, observations[i]]
    
    return (alpha, np.sum(alpha[N-1,:]))

def backward(params, observations):
    pi, A, O = params
    N = len(observations)
    S = pi.shape[0]
    
    beta = np.zeros((N, S))
    
    # base case
    beta[N-1, :] = 1
    
    # recursive case
    for i in range(N-2, -1, -1):
        for s1 in range(S):
            for s2 in range(S):
                beta[i, s1] += beta[i+1, s2] * A[s1, s2] * O[s2, observations[i+1]]
    
    return (beta, np.sum(pi * O[:, observations[0]] * beta[0,:]))

def baum_welch(training, pi, A, O, iterations):
    pi, A, O = np.copy(pi), np.copy(A), np.copy(O) # take copies, as we modify them
    S = pi.shape[0]
    
    # do several steps of EM hill climbing
    for it in range(iterations):
        pi1 = np.zeros_like(pi)
        A1 = np.zeros_like(A)
        O1 = np.zeros_like(O)
        
        for observations in training:
            # compute forward-backward matrices
            alpha, za = forward((pi, A, O), observations)
            beta, zb = backward((pi, A, O), observations)
            assert abs(za - zb) < 1e-6, "it's badness 10000 if the marginals don't agree"
            
            # M-step here, calculating the frequency of starting state, transitions and (state, obs) pairs
            pi1 += alpha[0,:] * beta[0,:] / za
            for i in range(0, len(observations)):
                O1[:, observations[i]] += alpha[i,:] * beta[i,:] / za
            for i in range(1, len(observations)):
                for s1 in range(S):
                    for s2 in range(S):
                        A1[s1, s2] += alpha[i-1,s1] * A[s1, s2] * O[s2, observations[i]] * beta[i,s2] / za
                                                                    
        # normalise pi1, A1, O1
        pi = pi1 / np.sum(pi1)
        for s in range(S):
            A[s, :] = A1[s, :] / np.sum(A1[s, :])
            O[s, :] = O1[s, :] / np.sum(O1[s, :])
    
    return pi, A, O

baum_welch(data, [1/7 for i in range(7)], transition, emission, 1000)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [137]:
transition, emission = baum_welch(data[:10], 1/7)
emission.shape

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide


(10, 10)

In [138]:
prior = 1/7
unique_characters = list(set(data))
transition = np.random.rand(7,7)
emission = np.random.rand(7,7)
for i in range(len(data)):
    alpha = forward(transition, emission, data, prior)
    beta = backward(transition, emission, data)
    g = gamma(transition, emission, data, prior)
    e = epsilon(transition, emission, data, prior)
    g2 = gamma(transition, emission, data, prior)
    print(e.shape)
    transition = tr(e, g)
    emission = em(g2, g)
    print(transition.shape, emission.shape)
    #print(transition.shape)
    break



(1000, 1000)
(1000, 1000) (1000, 1000)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in true_divide


# Study Guide

After the reading and the pre-class work you should be able to answer the following questions:
What is filtering (with reference to a hidden Markov model)? Give a compelling real-world example.
+ to find p(h_t|v_1:t)
What is smoothing (with reference to a hidden Markov model)? Give a compelling real-world example.
+ to find p(h_T|v_1:T)
What is prediction (with reference to a hidden Markov model)? Give a compelling real-world example.
+ p(v_t+1|v_t)
With reference to a Hidden Markov model, what is a transition matrix and what is an emission matrix?